
# HVMT Regression

This notebook builds a robust **Household VMT (HVMT)** model using NHTS 2017 → train and 2022 → test.  
It fixes common issues (column aliases, zero-VMT handling, year mismatches) and adds diagnostics:
- Clean loaders with **alias mapping** (e.g., `NUMVEH` ↔ `HHVEHCNT`)
- Create household `HH_VMT` from trip files and **merge**
- Optional filtering of **zero-VMT** households
- Train **Linear**, **Random Forest**, **XGBoost** (optionally **HistGradientBoosting** Poisson)
- **Smearing correction** back to original scale
- **Feature importances** (XGB), **residual plots**, and **segment analysis** (income, urban, vehicles)

> Expected inputs in the working directory (or update paths):  
> - `hhpub2017.csv`, `hhpub2022.csv` (household PUF)  
> - `trippub2017.csv`, `trippub2022.csv` (trip PUF)


In [1]:

# Cell 1: Imports & Config

import os
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

try:
    from xgboost import XGBRegressor
except Exception:
    XGBRegressor = None

# HistGradientBoosting for Poisson regression
try:
    from sklearn.ensemble import HistGradientBoostingRegressor
except Exception:
    HistGradientBoostingRegressor = None

import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 200)
RANDOM_STATE = 42


In [2]:
# Cell 2: Paths (edit these if your files are in another folder)

PATH_HH_2017 = "hhpub2017.csv"
PATH_HH_2022 = "hhpub2022.csv"
PATH_TR_2017 = "trippub2017.csv"
PATH_TR_2022 = "trippub2022.csv"


In [3]:

# Cell 3: Loaders with alias mapping and cleaning

# Canonical set used by modeling (you can add more later)
CANON = ["HHSIZE","NUMVEH","HHFAMINC","URBAN","HOMEOWN","HOMETYPE","HH_RACE","HH_HISP"]

# Aliases to normalize cross-year differences
ALIASES = {
    "HHSIZE":   ["HHSIZE","HH_SIZE","PERSONS","NUMPERSONS"],
    "NUMVEH":   ["HHVEHCNT","NUMVEH","VEHHO","HH_NUMVEH","VEH_CNT"],
    "HHFAMINC": ["HHFAMINC","HHINCOME","HHFAMINC_IMPUTED","HH_INC","FAMINCOME"],
    "URBAN":    ["URBAN","URBRUR","URBANSIZE","URBAN_RURAL","URBANCAT"],
    "HOMEOWN":  ["HOMEOWN","HOWND","TENURE","HH_OWN","HOME_OWN"],
    "HOMETYPE": ["HOMETYPE","HSTRUCT","UNITTYPE","BLTYP","HOME_TYPE"],
    "HH_RACE":  ["HH_RACE","HHRACE","R_RACE","RACE","RACE1"],
    "HH_HISP":  ["HH_HISP","HHHISP","HISPANIC","HISP","HISP1"],
}

SENTINELS = {-9, -8, -7, -1}

def normalize_household(path):
    df = pd.read_csv(path)
    if "HOUSEID" not in df.columns:
        raise KeyError(f"'HOUSEID' missing from {path}")
    # Build rename map by first matching alias per canonical
    present = set(df.columns)
    rename_map = {}
    for canon, choices in ALIASES.items():
        found = next((c for c in choices if c in present), None)
        if found:
            rename_map[found] = canon
    df = df.rename(columns=rename_map)
    keep_cols = ["HOUSEID"] + [c for c in CANON if c in df.columns]
    df = df[keep_cols].copy()
    # Replace sentinel values with NaN
    df = df.replace(list(SENTINELS), np.nan)
    return df

def trips_to_hh_vmt(path):
    t = pd.read_csv(path, usecols=["HOUSEID","TRPMILES"])
    t["TRPMILES"] = pd.to_numeric(t["TRPMILES"], errors="coerce")
    t.loc[t["TRPMILES"].isin(SENTINELS), "TRPMILES"] = np.nan
    t = t.dropna(subset=["TRPMILES"])
    hh = t.groupby("HOUSEID", as_index=False)["TRPMILES"].sum().rename(columns={"TRPMILES":"HH_VMT"})
    return hh


In [4]:

# Cell 4: Load, create HH_VMT, and merge

df_hh_2017 = normalize_household(PATH_HH_2017)
df_hh_2022 = normalize_household(PATH_HH_2022)

hh_vmt_2017 = trips_to_hh_vmt(PATH_TR_2017)
hh_vmt_2022 = trips_to_hh_vmt(PATH_TR_2022)

df_2017 = df_hh_2017.merge(hh_vmt_2017, on="HOUSEID", how="left")
df_2022 = df_hh_2022.merge(hh_vmt_2022, on="HOUSEID", how="left")

# Households with no recorded trips -> 0 VMT (common in NHTS)
df_2017["HH_VMT"] = df_2017["HH_VMT"].fillna(0.0)
df_2022["HH_VMT"] = df_2022["HH_VMT"].fillna(0.0)

print("2017 households:", len(df_2017), "| with VMT > 0:", int((df_2017['HH_VMT'] > 0).sum()))
print("2022 households:", len(df_2022), "| with VMT > 0:", int((df_2022['HH_VMT'] > 0).sum()))

# Quick peek
display_cols = ["HOUSEID","HH_VMT"] + [c for c in CANON if c in df_2017.columns][:6]
df_2017.head(3)[display_cols], df_2022.head(3)[display_cols]


2017 households: 129696 | with VMT > 0: 117192
2022 households: 7893 | with VMT > 0: 6152


(    HOUSEID   HH_VMT  HHSIZE  NUMVEH  HHFAMINC  URBAN  HOMEOWN  HH_RACE
 0  30000007  180.518       3       5       7.0      1      1.0      2.0
 1  30000008   16.034       2       4       8.0      4      1.0      1.0
 2  30000012  126.817       1       2      10.0      1      1.0      1.0,
       HOUSEID     HH_VMT  HHSIZE  NUMVEH  HHFAMINC  URBAN  HOMEOWN  HH_RACE
 0  9000013002  51.533872       4       2      11.0      1        1        1
 1  9000013016  21.866998       2       1       7.0      1        3        1
 2  9000013026   0.000000       1       0      10.0      1        3        1)

In [5]:

# Cell 5: Filter out zero-VMT households

FILTER_ZERO_VMT = True

if FILTER_ZERO_VMT:
    df_2017_nonzero = df_2017[df_2017["HH_VMT"] > 0].copy()
    df_2022_nonzero = df_2022[df_2022["HH_VMT"] > 0].copy()
else:
    df_2017_nonzero = df_2017.copy()
    df_2022_nonzero = df_2022.copy()

print(f"After filter -> 2017: {len(df_2017_nonzero)} | 2022: {len(df_2022_nonzero)}")


After filter -> 2017: 117192 | 2022: 6152


In [6]:

# Cell 6: Feature intersection & preprocessing

CANDIDATE_FEATURES = CANON.copy()
CAT_CANDS = ["HHFAMINC","URBAN","HOMEOWN","HOMETYPE","HH_RACE","HH_HISP"]
NUM_CANDS = ["HHSIZE","NUMVEH"]

present_2017 = [c for c in CANDIDATE_FEATURES if c in df_2017_nonzero.columns]
present_2022 = [c for c in CANDIDATE_FEATURES if c in df_2022_nonzero.columns]
COMMON_FEATURES = [c for c in CANDIDATE_FEATURES if c in present_2017 and c in present_2022]

if not COMMON_FEATURES:
    raise ValueError("No overlapping features between 2017 and 2022 after cleaning.")

cat_cols = [c for c in CAT_CANDS if c in COMMON_FEATURES]
num_cols = [c for c in NUM_CANDS if c in COMMON_FEATURES]

print("Using features:", COMMON_FEATURES)
print("Categorical:", cat_cols or "[]", "| Numeric:", num_cols or "[]")
print("Missing in 2017:", [c for c in CANDIDATE_FEATURES if c not in present_2017])
print("Missing in 2022:", [c for c in CANDIDATE_FEATURES if c not in present_2022])

# Prepare X/y
X_train = df_2017_nonzero[COMMON_FEATURES].copy()
X_test  = df_2022_nonzero[COMMON_FEATURES].copy()

for c in cat_cols:
    X_train[c] = X_train[c].astype("category")
    X_test[c] = X_test[c].astype("category")

y_train = np.log1p(df_2017_nonzero["HH_VMT"].clip(lower=0))
y_test  = np.log1p(df_2022_nonzero["HH_VMT"].clip(lower=0))
y_test_orig = df_2022_nonzero["HH_VMT"].values

# Preprocess
transformers = []
if cat_cols:
    transformers.append(("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_cols))
if num_cols:
    transformers.append(("num", StandardScaler(), num_cols))

pre = ColumnTransformer(transformers, remainder="drop")


Using features: ['HHSIZE', 'NUMVEH', 'HHFAMINC', 'URBAN', 'HOMEOWN', 'HH_RACE', 'HH_HISP']
Categorical: ['HHFAMINC', 'URBAN', 'HOMEOWN', 'HH_RACE', 'HH_HISP'] | Numeric: ['HHSIZE', 'NUMVEH']
Missing in 2017: ['HOMETYPE']
Missing in 2022: []


In [ ]:

# Cell 7: Train & evaluate models with smearing correction

models = {
    "Linear": LinearRegression(),
    "RF": RandomForestRegressor(n_estimators=300, random_state=RANDOM_STATE, n_jobs=-1),
}

if XGBRegressor is not None:
    models["XGB"] = XGBRegressor(
        n_estimators=400, max_depth=6, learning_rate=0.07,
        subsample=0.8, colsample_bytree=0.8, random_state=RANDOM_STATE, tree_method="hist"
    )

results = {}
for name, reg in models.items():
    pipe = Pipeline([("pre", pre), ("reg", reg)])
    pipe.fit(X_train, y_train)

    # Smearing correction using train residual variance
    y_pred_log_test  = pipe.predict(X_test)
    y_pred_log_train = pipe.predict(X_train)
    sigma2 = float(np.var(y_train - y_pred_log_train))
    y_pred_orig = np.expm1(y_pred_log_test + 0.5 * sigma2)

    # Metrics
    rmse = float(np.sqrt(mean_squared_error(y_test_orig, y_pred_orig)))
    mae  = float(mean_absolute_error(y_test_orig, y_pred_orig))
    r2   = float(r2_score(y_test_orig, y_pred_orig))

    # Record
    k_feats = int(pipe.named_steps["pre"].transform(X_train.head(1)).shape[1])
    results[name] = {"pipeline": pipe, "RMSE": rmse, "MAE": mae, "R2": r2, "sigma2": sigma2, "k_feats": k_feats}

for name, r in results.items():
    print(f"{name:<6} RMSE: {r['RMSE']:.2f}  MAE: {r['MAE']:.2f}  R²: {r['R2']:.4f}  (k={r['k_feats']})")

results


In [ ]:

# Cell 8: XGBoost feature importances and plot

def get_feature_names_from_column_transformer(ct):
    out = []
    for name, trans, cols in ct.transformers_:
        if name == "remainder":
            continue
        if hasattr(trans, "get_feature_names_out"):
            out.extend(list(trans.get_feature_names_out(cols)))
        else:
            out.extend(cols)
    return out

if "XGB" in results:
    xgb_pipe = results["XGB"]["pipeline"]
    feat_names = get_feature_names_from_column_transformer(xgb_pipe.named_steps["pre"])
    importances = xgb_pipe.named_steps["reg"].feature_importances_
    feat_imp_df = pd.DataFrame({"feature": feat_names, "importance": importances})                     .sort_values("importance", ascending=False)
    display(feat_imp_df.head(20))

    # Plot top 15
    topN = 15
    plt.figure(figsize=(8,6))
    feat_imp_df.head(topN).iloc[::-1].plot(kind="barh", x="feature", y="importance", legend=False)
    plt.title("Top XGBoost Feature Importances")
    plt.tight_layout()
    plt.show()
else:
    print("XGBoost not available. Skipping feature importances.")


In [ ]:

# Cell 9: Residual analysis (using best model by R²)

# Pick best model
best_name = max(results.items(), key=lambda kv: kv[1]["R2"])[0]
best_pipe = results[best_name]["pipeline"]

y_pred_log_test  = best_pipe.predict(X_test)
sigma2_best = results[best_name]["sigma2"]
y_pred_orig = np.expm1(y_pred_log_test + 0.5 * sigma2_best)

residuals = y_test_orig - y_pred_orig
abs_err = np.abs(residuals)

print(f"Best model: {best_name}")
print(pd.Series(residuals).describe())

# Residual vs. predicted plot
plt.figure(figsize=(6,5))
plt.scatter(y_pred_orig, residuals, alpha=0.3, s=8)
plt.axhline(0, linestyle="--")
plt.xlabel("Predicted HH_VMT")
plt.ylabel("Residual (y - y_pred)")
plt.title("Residuals vs Predicted")
plt.tight_layout()
plt.show()

# Residual distribution
plt.figure(figsize=(6,5))
plt.hist(residuals, bins=50)
plt.title("Residual Distribution")
plt.xlabel("Residual")
plt.ylabel("Count")
plt.tight_layout()
plt.show()


In [ ]:

# Cell 10: Segment analysis (income, urban, vehicle count)

anal_df = df_2022_nonzero.copy()
anal_df = anal_df.assign(y_true=y_test_orig, y_pred=y_pred_orig, abs_err=abs_err)

def metric_table(df, key):
    g = df.groupby(key).agg(
        n=("HOUSEID","count"),
        rmse=("abs_err", lambda x: np.sqrt(np.mean((x**2)))),
        mae=("abs_err","mean")
    ).reset_index().sort_values("rmse")
    return g

# Income
if "HHFAMINC" in anal_df.columns:
    print("By HHFAMINC:")
    display(metric_table(anal_df, "HHFAMINC").head(20))

# Urban
if "URBAN" in anal_df.columns:
    print("By URBAN:")
    display(metric_table(anal_df, "URBAN"))

# Vehicles
if "NUMVEH" in anal_df.columns:
    bins = pd.cut(anal_df["NUMVEH"], bins=[-0.1,0.5,1.5,2.5,10], labels=["0","1","2","3+"])
    anal_df["NUMVEH_BIN"] = bins
    print("By NUMVEH_BIN:")
    display(metric_table(anal_df, "NUMVEH_BIN"))


In [ ]:

# Cell 11 : HistGradientBoosting with Poisson loss on original scale
# Poisson is often better for non-negative, skewed targets (like miles).

if HistGradientBoostingRegressor is not None:
    # Train on original (non-log) target
    y_train_poiss = df_2017_nonzero["HH_VMT"].clip(lower=0)
    y_test_poiss  = df_2022_nonzero["HH_VMT"].clip(lower=0)

    # Reuse preprocessing
    pipe_poiss = Pipeline([("pre", pre),
                           ("reg", HistGradientBoostingRegressor(loss="poisson",
                                                                 learning_rate=0.07,
                                                                 max_depth=6,
                                                                 max_iter=500,
                                                                 random_state=RANDOM_STATE))])
    pipe_poiss.fit(X_train, y_train_poiss)
    y_pred_poiss = pipe_poiss.predict(X_test)

    rmse = float(np.sqrt(mean_squared_error(y_test_poiss, y_pred_poiss)))
    mae  = float(mean_absolute_error(y_test_poiss, y_pred_poiss))
    r2   = float(r2_score(y_test_poiss, y_pred_poiss))
    print(f"HGB-Poisson  RMSE: {rmse:.2f}  MAE: {mae:.2f}  R²: {r2:.4f}")
else:
    print("HistGradientBoostingRegressor not available. Skipping Poisson test.")
